In [1]:
import pandas as pd 
import numpy as np

# Load the datasets
df1=pd.read_csv('../input/tmdb-movie-metadata/tmdb_5000_credits.csv')
df2=pd.read_csv('../input/tmdb-movie-metadata/tmdb_5000_movies.csv')

In [2]:
# Join the two dataset on the 'id' column
df1.columns = ['id','tittle','cast','crew']
df2= df2.merge(df1,on='id')

In [3]:
df2.head()

budget                        ...                                                                       crew
0  237000000                        ...                          [{"credit_id": "52fe48009251416c750aca23", "de...
1  300000000                        ...                          [{"credit_id": "52fe4232c3a36847f800b579", "de...
2  245000000                        ...                          [{"credit_id": "54805967c3a36829b5002c41", "de...
3  250000000                        ...                          [{"credit_id": "52fe4781c3a36847f81398c3", "de...
4  260000000                        ...                          [{"credit_id": "52fe479ac3a36847f813eaa3", "de...

[5 rows x 23 columns]

Demographic Filtering
---------------------

In [4]:
C= df2['vote_average'].mean()
C

6.092171559442011

In [5]:
m= df2['vote_count'].quantile(0.9)
m

1838.4000000000015

In [6]:
q_movies = df2.copy().loc[df2['vote_count'] >= m]
q_movies.shape

(481, 23)

In [7]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [8]:
# Define a new feature 'score' and calculate its value with `weighted_rating()`
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

In [9]:
import matplotlib.pyplot as plt

In [10]:
# Sort movies based on score calculated above
q_movies = q_movies.sort_values('score', ascending=False)

# Print the top 15 movies
q_movies[['title', 'vote_count', 'vote_average', 'score', 'popularity']].head(15)

title     ...      popularity
1881                           The Shawshank Redemption     ...      136.747729
662                                          Fight Club     ...      146.757391
65                                      The Dark Knight     ...      187.322927
3232                                       Pulp Fiction     ...      121.463076
96                                            Inception     ...      167.583710
3337                                      The Godfather     ...      143.659698
95                                         Interstellar     ...      724.247784
809                                        Forrest Gump     ...      138.133331
329       The Lord of the Rings: The Return of the King     ...      123.630332
1990                            The Empire Strikes Back     ...       78.517830
262   The Lord of the Rings: The Fellowship of the Ring     ...      138.049577
2912                                          Star Wars     ...      126.393695
1818                                   Schindler's List     ...      104.469351
3865                                           Whiplash     ...      192.528841
330               The Lord of the Rings: The Two Towers     ...      106.914973

[15 rows x 5 columns]

In [11]:
plt.figure(figsize=(12,4))

plt.barh(q_movies['title'].head(15), q_movies['score'].head(15), align='center', color='orange')
plt.gca().invert_yaxis()
plt.xlabel("Score")
plt.title("Trending Movies")

Text(0.5,1,'Trending Movies')

In [12]:
pop= df2.sort_values('popularity', ascending=False)

plt.figure(figsize=(12,4))

plt.barh(pop['title'].head(15), pop['popularity'].head(15), align='center', color='lightgreen')
plt.gca().invert_yaxis()
plt.xlabel("Popularity")
plt.title("Popular Movies")

Text(0.5,1,'Popular Movies')

Content Based Filtering
-----------------------

In this recommender system the content of the movie (overview, cast, crew, keyword, tagline etc) is used to find its similarity with other movies. Then the movies that are most likely to be similar are recommended.

In [13]:
df2['overview'].head(10)

0    In the 22nd century, a paraplegic Marine is di...
1    Captain Barbossa, long believed to be dead, ha...
2    A cryptic message from Bond’s past sends him o...
3    Following the death of District Attorney Harve...
4    John Carter is a war-weary, former military ca...
5    The seemingly invincible Spider-Man goes up ag...
6    When the kingdom's most wanted-and most charmi...
7    When Tony Stark tries to jumpstart a dormant p...
8    As Harry begins his sixth year at Hogwarts, he...
9    Fearing the actions of a god-like Super Hero l...
Name: overview, dtype: object

In [14]:
# Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

# Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#,Replace NaN with an empty string
df2['overview'] = df2['overview'].fillna('')

# Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df2['overview'])

# Output the shape of tfidf_matrix
tfidf_matrix.shape

(4803, 20978)

In [15]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [16]:
# Construct a reverse map of indices and movie titles
indices = pd.Series(df2.index, index=df2['title']).drop_duplicates()

In [17]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    try:
        # Get the index of the movie that matches the title
        idx = indices[title]
    except:
        return 'Cannot find the index of the movie that matches the title'

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    try:
        # Sort the movies based on the similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    except:
        pass

    # Get the scores of the 10 most similar movies
    if len(sim_scores) > 11:
        sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df2[['title', 'release_date', 'id']].iloc[movie_indices]

In [18]:
get_recommendations('Spider-Man')

title release_date      id
5                  Spider-Man 3   2007-05-01     559
38     The Amazing Spider-Man 2   2014-04-16  102382
20       The Amazing Spider-Man   2012-06-27    1930
30                 Spider-Man 2   2004-06-25     558
1534              Arachnophobia   1990-08-09    6488
953   Gremlins 2: The New Batch   1990-06-15     928
1720                   Kick-Ass   2010-03-22   23483
2740                The New Guy   2002-05-10   10985
3216                   Election   1999-04-23    9451
1572  Forgetting Sarah Marshall   2008-04-17    9870

In [19]:
get_recommendations('Superman')

title release_date      id
870                        Superman II   1980-12-04    8536
10                    Superman Returns   2006-06-28    1452
2433  Superman IV: The Quest for Peace   1987-07-23   11411
914               Central Intelligence   2016-06-15  302699
4659                        Horse Camp   2015-01-10  306667
2527       National Lampoon's Vacation   1983-07-28   11153
3542                              R100   2013-10-05  193722
45                         World War Z   2013-06-20   72190
1368                    Identity Thief   2013-02-07  109431
1237                The Out-of-Towners   1999-04-02    8970